In [1]:
#Cutting Stock Problem
#Column generation method
from gurobipy import *
from random import randint
from math import floor

In [2]:
#不同宽度产品数量
k=50
#基础材料数量
L=750
#每一个产品的长度或者宽度
l={i:randint(10,floor(L/3)) for i in range(k)}
#每个产品的需求
d={i:randint(5,200) for i in range(k)}
#----Pricing Model-----
PP=Model("Pricing Problem")
Pvars=PP.addVars(k,obj=-1,vtype=GRB.INTEGER,name='w')
PP.addConstr(Pvars.prod(l)<=L)
PP.update()


In [3]:
#----Master Model-----
M=Model('Master Problem')
s=M.addVars(k,name='slack',obj=1000)
c=M.addConstrs(s[i]>=d[i] for i in range(k))
M.write('initial.lp')
M.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 50 rows, 50 columns and 50 nonzeros
Model fingerprint: 0x2244a41b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+02]
Presolve removed 50 rows and 50 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.8590000e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.859000000e+06


In [4]:
#---Adding a set of initial variables---
PP.Params.OutputFlag=0
for i in range(k):
    obj={i:-1 for i in range(k)}
    obj[i]=-L-1
    PP.setObjective(Pvars.prod(obj))
    PP.optimize()
    col=Column()
    for j in range(k):
        col.addTerms(Pvars[j].x,c[j])
    M.addVar(obj=1,column=col)
M.optimize()
M.write('base.lp')

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 50 rows, 100 columns and 140 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.0234238e+35   5.728125e+31   4.023424e+05      0s
      50    8.7898998e+02   0.000000e+00   0.000000e+00      0s

Solved in 50 iterations and 0.06 seconds (0.00 work units)
Optimal objective  8.789899768e+02


In [5]:
#----
M.Params.OutputFlag=0
Iter=0
while M.Status==GRB.OPTIMAL:
    pi={i:-c[i].pi for i in range(k)}
    PP.setObjective(Pvars.prod(pi))
    PP.optimize()
    if PP.Status!=GRB.OPTIMAL:
        raise('Unexpected optimization status')
    if PP.ObjVal> -1.0001:
        break
    if Iter % 20 ==0:
        print('Iteration MasterValue Pricing Value')
    print('%8d %12.5g %12.5g' % (Iter,M.ObjVal,PP.ObjVal))
    Iter+=1

    col=Column()
    for j in range(k):
        col.addTerms(Pvars[j].x,c[j])
    M.addVar(obj=1,column=col)
    M.optimize()
M.write('final.lp')
rootbound=M.ObjVal

Iteration MasterValue Pricing Value
       0       878.99      -1.2633
       1       875.65      -1.2267
       2        865.7      -1.2051
       3       857.16         -1.2
       4       856.56      -1.2143
       5       854.66         -1.2
       6       854.33      -1.1778
       7       852.74      -1.1667
       8       839.81      -1.1667
       9       837.96      -1.1667
      10       830.76      -1.1562
      11       828.47      -1.1562
      12       823.55      -1.1429
      13       812.81      -1.1429
      14       810.92      -1.1429
      15       808.93      -1.1498
      16       806.93      -1.1429
      17       805.78      -1.1429
      18       803.56       -1.119
      19       803.53      -1.1167
Iteration MasterValue Pricing Value
      20       803.02      -1.1125
      21       798.23      -1.1026
      22       797.58      -1.0897
      23       796.86      -1.0897
      24          795        -1.08
      25       793.75      -1.0755
      26        79

In [6]:

#-----
M.Params.OutputFlag=1
Mvars=M.getVars()
for v in Mvars:
    v.vtype=GRB.INTEGER
M.optimize()
print('Proven column generation gap %.3f%%' % (100*(M.ObjVal-rootbound)/M.ObjVal))

Set parameter OutputFlag to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 50 rows, 208 columns and 477 nonzeros
Model fingerprint: 0x01d3385c
Variable types: 0 continuous, 208 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 2e+02]
Found heuristic solution: objective 1396.0000000
Presolve removed 2 rows and 53 columns
Presolve time: 0.00s
Presolved: 48 rows, 155 columns, 423 nonzeros
Variable types: 0 continuous, 155 integer (2 binary)

Root relaxation: objective 9.413640e+02, 106 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  941.36400    0   46 1396.00000  941.36400  32.6%     -    0s
H